# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [34]:
from selenium import webdriver # importando apenas o webdriver do selenium
from selenium.webdriver.common.keys import Keys

navegador = webdriver.Chrome() # Aqui vamos abrir o navegador controlado pela biblioteca selenium

# Passo 1: Pegar a cotação do dólar
navegador.get("https://www.google.com/") # entrar no google

# xpath = identifica oque é cada id, cada coisa em uma página -> DICA: usar sempre aspas simples no xpath ''
#navegador.find_element(o elemento).oque eu quero fazer
    # .click
    # .get
    # .send_keys

# Escrevendo cotação dólar na barra de busca do google
navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação dólar")

# Dando enter
navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

# Pegando uma informação
cotacao_dolar = navegador.find_element('xpath', '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

print(cotacao_dolar)

# Passo 2: Pegar a cotação do euro
navegador.get("https://www.google.com/")

navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação euro")

navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_euro = navegador.find_element('xpath', '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

print(cotacao_euro)

# Passo 3: Pegar a cotação do ouro
navegador.get("https://www.melhorcambio.com/ouro-hoje")

cotacao_ouro = navegador.find_element('xpath', '//*[@id="comercial"]').get_attribute('value')
cotacao_ouro = cotacao_ouro.replace(",", ".") # Mudou a virgula da cotação do ouro para .

print(cotacao_ouro)

navegador.quit()

5.16
5.556675
311.97


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [35]:
# Passo 4: Importar a base de dados e atualizar a base
import pandas as pd

tabela = pd.read_excel("Produtos.xlsx")
display(tabela)



,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [36]:
# Passo 5: Recalcular os preços

# atualizar a cotação
# cotacao dolar

tabela.loc[tabela["Moeda"] == "Dólar", "Cotação"] = float(cotacao_dolar)
tabela.loc[tabela["Moeda"] == "Euro", "Cotação"] = float(cotacao_euro)
tabela.loc[tabela["Moeda"] == "Ouro", "Cotação"] = float(cotacao_ouro)

# preco de compra = cotação * preco original
tabela['Preço de Compra'] = tabela["Cotação"] * tabela["Preço Original"]

# preco de venda = preco de compra * margem
tabela['Preço de Venda'] = tabela["Preço de Compra"] * tabela["Margem"]

display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.160000,5159.9484,1.40,7223.92776
1,Carro Renault,4500.00,Euro,5.556675,25005.0375,2.00,50010.07500
2,Notebook Dell,899.99,Dólar,5.160000,4643.9484,1.70,7894.71228
3,IPhone,799.00,Dólar,5.160000,4122.8400,1.70,7008.82800
4,Carro Fiat,3000.00,Euro,5.556675,16670.0250,1.90,31673.04750
5,Celular Xiaomi,480.48,Dólar,5.160000,2479.2768,2.00,4958.55360
6,Joia 20g,20.00,Ouro,311.970000,6239.4000,1.15,7175.31000


### Agora vamos exportar a nova base de preços atualizada

In [37]:
# Passo 6: Exportar a base atualizada
tabela.to_excel("Produtos Novo.xlsx", index=False)